# Проект e-commerce
*Анализ совершенных клиентами сервиса покупок*

## Содержание
**I.** Сколько пользователей, которые совершили покупку только один раз?

**II.** Сколько заказов в месяц в среднем не доставляется по разным причинам? 

**III.** По каждому товару определить, в какой день недели товар чаще всего покупается.

**IV.** Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?

**V.** Проведите ***когортный анализ*** пользователей. В период с января по декабрь выявите когорту с самым высоким *retention* на 3й месяц.

**VI.** Используя python, постройте ***RFM-сегментацию*** пользователей, чтобы качественно оценить свою аудиторию.

In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

from datetime import timedelta

In [2]:
customers_df   = pd.read_csv('olist_customers_dataset.csv')

In [3]:
orders_df      = pd.read_csv('olist_orders_dataset.csv')

In [4]:
order_items_df = pd.read_csv('olist_order_items_dataset.csv')

### Обзор данных и работа с типами

In [5]:
customers_df.shape

(99441, 5)

In [6]:
customers_df.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [7]:
customers_df.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [8]:
orders_df.shape

(99441, 8)

In [9]:
orders_df.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [10]:
orders_df.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [11]:
columns = [
            'order_purchase_timestamp', 
            'order_approved_at', 
            'order_delivered_carrier_date', 
            'order_delivered_customer_date',
            'order_estimated_delivery_date'
]

for column in columns:
    orders_df[column] = pd.to_datetime(orders_df[column])

In [12]:
order_items_df.shape

(112650, 7)

In [13]:
order_items_df.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [14]:
order_items_df.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [15]:
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

In [16]:
# количество доставленных заказов
delivered_orders = orders_df.query('order_status == "delivered"')

delivered_orders.order_id.nunique()

96478

Определимся с понятием "покупка", чтобы однозначно проводить дальнейший анализ.

>Покупка - те заказы, статус которых "delivered" (доставлен пользователю), так как это указывает на фактическое получение пользователем товара. Таким образом, покупок всего 96478

## I. Сколько пользователей, которые совершили покупку только один раз?

In [17]:
# объединение датафреймов по ключу 'customer_id'
customers_and_orders_df = orders_df.merge(customers_df, on='customer_id')

In [18]:
# подсчет количества покупок каждого пользователя
customer_orders = customers_and_orders_df.query('order_status == "delivered"') \
                    .groupby('customer_unique_id') \
                    [['order_id']].nunique()

In [19]:
# какие бывают уникальные количетсва покупок
customer_orders.order_id.unique()

# фильтрация - отбираем пользователей с одной покупкой
one_order_customers = customer_orders.query('order_id == 1')

print('Количество пользователей, которые совершили покупку один раз =', one_order_customers.shape[0])

Количество пользователей, которые совершили покупку один раз = 90557


# II. Сколько заказов в месяц в среднем не доставляется по разным причинам? 


In [20]:
orders_df.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

Причин не доставки заказов всего две - canceled (отменен), unavailable (недоступен).

In [21]:
# добавление колонки месяца в датасет
orders_df['order_month'] = orders_df['order_purchase_timestamp'].dt.month

# отбор заказов с недоставленным статусом
canceled_orders    = orders_df.query('order_status == "canceled"')
unavailable_orders = orders_df.query('order_status == "unavailable"')

# подсчет среднего
avg_canceled_orders_by_month    = canceled_orders.groupby('order_month').agg({'order_status': 'count'}).mean().order_status.round(0)
avg_unavailable_orders_by_month = unavailable_orders.groupby('order_month').agg({'order_status': 'count'}).mean().order_status.round(0)


# детализация
print('Отменены по причние "canceled" =', canceled_orders.groupby('order_month').agg({'order_status': 'count'}).mean().order_status.round(0))
print('Отменены по причине "unavailable" =', unavailable_orders.groupby('order_month').agg({'order_status': 'count'}).mean().order_status.round(0))

Отменены по причние "canceled" = 52.0
Отменены по причине "unavailable" = 51.0


## III. По каждому товару определить, в какой день недели товар чаще всего покупается.


In [25]:
merged_df = orders_df.merge(order_items_df, on='order_id')

In [28]:
merged_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_month,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,10,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,7,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,8,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,11,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [38]:
merged_df['day_of_the_week'] = merged_df.order_purchase_timestamp.dt.dayofweek

In [41]:
order_by_dow = merged_df.query('order_status == "delivered"') \
                            .groupby(['order_id', 'day_of_the_week'], as_index=False) \
                            .agg({'customer_id' : 'count'}) \
                            .rename(columns={'customer_id' : 'count'}) \
                            .sort_values('count', ascending=False)

In [46]:
order_by_dow.groupby('order_id').head(1)

,order_id,day_of_the_week,count
49031,8272b63d03f5f79c56e9e4120aec44ef,6,21
64273,ab14fdcfbe524636d65ee38360e22ce8,2,20
10239,1b15974a0141d54e36626dca3fdc731a,3,20
59603,9ef13efd6949e4573a18964dd1bbe7f5,0,15
25024,428a2f660dc84138d969ccd69a0ab6d5,3,15
...,...,...,...
33871,5a0911d70c1f85d3bed0df1bf693a6dd,3,1
33870,5a082b558a3798d3e36d93bfa8ca1eae,0,1
33868,5a07264682e0b8fbb3f166edbbffc6e8,3,1
33867,5a071192a28951b76774e5a760c8c9b7,3,1


## **IV.** Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?

In [47]:
# Объединение датафреймов по столбцу customer_id
merged_df =  customers_df.merge(orders_df, on='customer_id')

# Создание колонки с неделей и месяцем
merged_df['week_period'] = merged_df['order_purchase_timestamp'].dt.to_period('W')
merged_df['month'] = merged_df['order_purchase_timestamp'].dt.to_period('M')

# Расчёт количества недель в каждом месяце
week_per_month = merged_df.groupby('month', as_index=False)['week_period'].nunique()

# Группировка данных по id клиента, неделе, месяцу и расчёт количества заказов для каждой недели
orders_per_week = merged_df.groupby(['customer_id', 'week_period', 'month'], as_index=False) \
    .agg({'order_id': 'count'})

# Объединение данных с количеством недель в каждом месяце
orders_per_week = orders_per_week.merge(week_per_month, on='month')

# Расчёт среднего количества заказов в месяц путём произведения количества заказов и недель в каждом месяце
orders_per_week['avg_orders_per_week'] = orders_per_week['order_id'] / orders_per_week['week_period_y']

orders_per_week[['customer_id', 'month', 'avg_orders_per_week']]

,customer_id,month,avg_orders_per_week
0,00012a2ce6f8dcda20d059ce98491703,2017-11,0.2
1,000f17e290c26b28549908a04cfe36c1,2017-11,0.2
2,001574cd5824c0b1ea90dd4f4ba6d5b8,2017-11,0.2
3,0026d252429f669d454d726e5e956aa1,2017-11,0.2
4,002937abdae13680e17dccd3868b4825,2017-11,0.2
...,...,...,...
99436,0975c3cf2e418ebfa5d1191a2dffd59d,2016-12,1.0
99437,4c2ec60c29d10c34bd49cb88aa85cfc4,2018-10,0.5
99438,856336203359aa6a61bf3826f7d84c49,2018-10,0.5
99439,a4b417188addbc05b26b72d5e44837a1,2018-10,0.5


## **V.** Проведите ***когортный анализ*** пользователей. В период с января по декабрь выявите когорту с самым высоким *retention* на 3й месяц.


In [52]:
 # Объединение датафреймов по столбцу customer_id
merged_df =  customers_df.merge(orders_df, on='customer_id')

# Создание колонки с месяцем
merged_df['month'] = merged_df['order_purchase_timestamp'].dt.to_period('M')

# Фильтрация данных по периоду с января по декабрь 2017 года
df_2017 = merged_df.loc[merged_df['month'].between('2017-01', '2017-12')]

# Группировка данных по штатам и месяцам заказа и вычисление размера каждой когорты, а также преобразование таблицы
cohorts_data = df_2017.groupby(['customer_state', 'month']).agg({'customer_unique_id': 'nunique'}).unstack()

# Вычисление retention на 3-й месяц путём деления 3-го месяца на первый
third_month_retention = cohorts_data.iloc[:, 2] / cohorts_data.iloc[:, 0]

# Отбор котогорты с самым высоким retention
result = third_month_retention.idxmax()

print("Когорта с самым высоким retention на 3-й месяц:", result)

Когорта с самым высоким retention на 3-й месяц: MS
